# First steps in PySpark 

In this notebook we will learn the fundamentals of functional programming, as well as the basic abstraction of a distributed object in Spark, the RDD. The notebook has been divided into two parts:

Part 1: map/reduce basics

Part 2: Work with RDD and Pair RDD abstractions 






# Part 1: map/reduce basics

![Hadoop Logo](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Hadoop_logo.svg/220px-Hadoop_logo.svg.png)
# **Apache Hadoop (MapReduce)**

It is an open source software framework written in Java for distributed storage and distributed processing of very large data sets on computer clusters built from commodity hardware. All the modules in Hadoop are designed with a fundamental assumption that hardware failures (of individual machines, or racks of machines) are common and thus should be automatically handled in software by the framework.

The core of Apache Hadoop consists of a storage part (Hadoop Distributed File System (HDFS)) and a processing part (MapReduce). Hadoop splits files into large blocks and distributes them amongst the nodes in the cluster. To process the data, Hadoop MapReduce transfers packaged code for nodes to process in parallel, based on the data each node needs to process. This approach takes advantage of data locality — nodes manipulating the data that they have on hand — to allow the data to be processed faster and more efficiently than it would be in a more conventional supercomputer architecture that relies on a parallel file system where computation and data are connected via high-speed networking.

![caption](http://d152j5tfobgaot.cloudfront.net/wp-content/uploads/2012/07/mapreduce.png)

Since data and computation are distributed, we should avoid the use of variables, i.e. mutable data. Thus, in contrast to impertaive programming, we shall use the functional approach (lambda calculus).

### The goal of the following excercises is to understand basic lambda calculus with python.

### (1a) Functional programming in Python

So, what is Functional Programming? From Wikipedia: 

« …a  programing paradigm that treats computation as the evaluation of mathematical functions and **avoids changing-state and mutable  data**.»

It´s based upon Lambda calculus, wich consist of:
 * Function definition (declaration of expressions)
 * Function application (evaluation of those expressions)
 * Recursion (iteration)

We have already used this in python!!! :)

Recall the typical "lambda x: x+1" we have been using as the first argument of map, reduce and filter methods:
 * **map** maps each value in the input collection to a different value. It´s just the classical mathematical funciton we are used to!
 * **reduce** takes two values from the input collection and returns a new value (of the same type) by appliying a commutative operation to them. 
 * **filter** filters the elements in the input collection according to a certain (boolean) criteria.
 

**Mapping**

![map](https://cosminpupaza.files.wordpress.com/2015/10/map.png?w=505)

In [3]:
x = [6,4,1,3,6,6,3,34,7,4]
list(map(lambda i: i**2, x))

[36, 16, 1, 9, 36, 36, 9, 1156, 49, 16]

In [6]:
l = 'Hola buenos dias estoy separando por espacios para hacer una lista'.split()
len(l)

11

In [8]:
list(map(lambda w: len(w), l))

[4, 6, 4, 5, 9, 3, 8, 4, 5, 3, 5]

In [12]:
list(map(lambda i: i%2, x))

[0, 0, 1, 1, 0, 0, 1, 0, 1, 0]

**Filtering**
![filter](https://cosminpupaza.files.wordpress.com/2015/11/filter.png?w=405)

In [14]:
list(filter(lambda i: i%2==0, x))

[6, 4, 6, 6, 34, 4]

In [15]:
list(filter(lambda i: i%2!=0, x))

[1, 3, 3, 7]

In [16]:
list(filter(lambda w: len(w)>4, l))

['buenos', 'estoy', 'separando', 'espacios', 'hacer', 'lista']

**Reducing** Recall it must be commutative! Think about the importance of this when parallelizing computations

![](https://cosminpupaza.files.wordpress.com/2015/11/reduce.png?w=500)

In [18]:
m = list(map(lambda i: i%2, x))

In [19]:
m

[0, 0, 1, 1, 0, 0, 1, 0, 1, 0]

In [21]:
from functools import reduce

In [24]:
reduce(lambda acc, i: acc+i, m)

4

In [25]:
x

[6, 4, 1, 3, 6, 6, 3, 34, 7, 4]

In [29]:
reduce(lambda acc, i: acc+i, filter(lambda i: i==1, map(lambda i: i%2, x)))

4

## (1b) Exercise: Calculate the mean of a collection of real numbers using map/reduce
Recall:

$$\bar x = \frac{\sum_{i=1}^{N} x_i}{N} $$

It´s straightforward to do this with python built-in mehots sum() and len(). However, how would you do that with map/reduce? We have already shown how to sum the elements of an array. Thus, you have to calculate the length of the array. For this:
 * Create another array of the same size, consisting of 1s.
 * Sum the elements of that array

In [30]:
suma = reduce(lambda acc,i: acc+i, x)

In [31]:
suma

74

In [32]:
len(x)

10

In [33]:
reduce(lambda acc, i: acc+1, x)

15

In [34]:
x

[6, 4, 1, 3, 6, 6, 3, 34, 7, 4]

In [40]:
count = reduce(lambda acc, i: acc+i, map(lambda i: 1, x))

In [41]:
mean = suma/count
mean

7.4

## (1c) Exercise: Calculate the standard deviation of a collection of real numbers
Recall:

$$\sigma_x^2 = \frac{\sum_{i=1}^{N} (x_i-\bar x)^2}{N-1} =
\frac{\sum_{i=1}^{N} (x_i^2+{\bar x}^2-2x_i\bar x)}{N-1} =
\frac{1}{N-1}\left(\sum_i x_i^2-N\bar x^2\right)$$

For this, use the *mean* and *count* variables from the previous excercise.

In [42]:
x

[6, 4, 1, 3, 6, 6, 3, 34, 7, 4]

In [60]:
reduce(lambda acc,i: acc+i, map(lambda i: (i-mean)**2, x))/count

81.64000000000001

In [61]:
(reduce(lambda acc,i: acc+i, map(lambda i: i**2, x))-count*mean**2)/count

81.64

In [54]:
reduce(lambda acc, i: acc + i**2, x) # mal

1334

In [52]:
reduce(lambda acc,i: acc+i, map(lambda i: i**2, x))

1364

In [56]:
import numpy as np

In [62]:
np.var(x)

81.640000000000015

## (1c.bis) Exercise: all at once! 
For the std calculation, we have obtained separatedly the sum of elements, the lenght and the sum of the elements squared. That is, we have swept the array three times! Can you do it in a two step process using map/reduce? Do you think it might matter at some point?
 * Hint: recall that reduce takes two arguments of the same type, and returns another value of that type. So, instead of using numbers as the elements of our array, use tuples!!

In [63]:
x

[6, 4, 1, 3, 6, 6, 3, 34, 7, 4]

In [64]:
list(map(lambda i: (i, 1, i**2), x))

[(6, 1, 36),
 (4, 1, 16),
 (1, 1, 1),
 (3, 1, 9),
 (6, 1, 36),
 (6, 1, 36),
 (3, 1, 9),
 (34, 1, 1156),
 (7, 1, 49),
 (4, 1, 16)]

In [66]:
reduce(lambda acc,i: acc+i, map(lambda i: (i, 1, i**2), x)) # casi! ha concatenado, no sumado, as'i que est'a mal

(6,
 1,
 36,
 4,
 1,
 16,
 1,
 1,
 1,
 3,
 1,
 9,
 6,
 1,
 36,
 6,
 1,
 36,
 3,
 1,
 9,
 34,
 1,
 1156,
 7,
 1,
 49,
 4,
 1,
 16)

In [67]:
def sumaTuplas(acc, t):
    num, uno, cuad = t
    num_acc, uno_acc, cuad_acc = acc
    
    return (num_acc+num, uno+uno_acc, cuad+cuad_acc)

In [72]:
tuplaAgregada = reduce(sumaTuplas, map(lambda i: (i, 1, i**2), x))
tuplaAgregada

(74, 10, 1364)

In [81]:
suma = tuplaAgregada[0]
count = tuplaAgregada[1]
sumacuads = tuplaAgregada[2]

In [84]:
suma, count, sumacuads = tuplaAgregada  # una manera m'as compacta de hacer lo mismo que la celda anterior

In [85]:
mean = suma/count
mean

7.4

In [86]:
(sumacuads - count*mean**2)/count

81.64

## (1d) Twe 'word-count' problem: creating histograms
Given a set of keys in an input collection, calculate the frequency of each key. 

In order to understand better how map/reduce works, we will implement this simple calculation in several forms.

For simplicity, we are going to create a list of numbers between 1 and 9, that can be repeated a (random) number of times.

In [87]:
import random

random.seed(42)
a = [random.randint(1, 9) for _ in range(500)]
a

[2,
 1,
 5,
 4,
 4,
 3,
 2,
 9,
 2,
 7,
 1,
 1,
 2,
 4,
 4,
 9,
 1,
 9,
 4,
 9,
 7,
 4,
 8,
 5,
 1,
 3,
 7,
 6,
 5,
 3,
 4,
 6,
 2,
 2,
 7,
 2,
 6,
 6,
 5,
 1,
 8,
 9,
 2,
 7,
 2,
 9,
 5,
 6,
 4,
 2,
 1,
 4,
 5,
 2,
 4,
 2,
 7,
 5,
 8,
 6,
 3,
 6,
 6,
 4,
 5,
 2,
 3,
 9,
 4,
 3,
 8,
 7,
 5,
 9,
 4,
 6,
 1,
 4,
 1,
 6,
 7,
 5,
 2,
 4,
 6,
 4,
 8,
 7,
 8,
 3,
 5,
 3,
 4,
 9,
 9,
 5,
 7,
 7,
 6,
 4,
 3,
 9,
 8,
 2,
 1,
 2,
 3,
 3,
 7,
 2,
 7,
 7,
 8,
 9,
 5,
 9,
 1,
 2,
 9,
 5,
 6,
 2,
 5,
 7,
 3,
 8,
 1,
 5,
 9,
 3,
 9,
 2,
 5,
 9,
 4,
 3,
 6,
 3,
 9,
 9,
 1,
 6,
 8,
 1,
 2,
 6,
 5,
 4,
 1,
 4,
 2,
 2,
 8,
 2,
 9,
 3,
 3,
 8,
 9,
 3,
 5,
 9,
 7,
 4,
 9,
 4,
 5,
 7,
 6,
 8,
 9,
 8,
 2,
 4,
 4,
 2,
 6,
 1,
 9,
 4,
 4,
 1,
 2,
 1,
 4,
 2,
 1,
 6,
 2,
 9,
 4,
 5,
 8,
 4,
 9,
 3,
 8,
 4,
 8,
 7,
 4,
 2,
 2,
 7,
 6,
 7,
 7,
 8,
 1,
 2,
 1,
 7,
 6,
 2,
 4,
 4,
 4,
 9,
 8,
 3,
 7,
 3,
 5,
 8,
 4,
 2,
 8,
 9,
 2,
 1,
 9,
 1,
 2,
 4,
 3,
 7,
 8,
 8,
 4,
 7,
 1,
 3,
 7,
 1,
 7,
 5,
 8,
 5,
 7,
 9,


### (1d.1) Simple approach

 * Start with an empty dict
 * If a new key is not present in the dict, create it.
 * Otherwise, increase the frequency of the key by one.

In [88]:
d = {}

for i in a:
    if i in d:
        d[i] +=1
    else:
        d[i] = 1
        
d

{1: 52, 2: 64, 3: 49, 4: 70, 5: 55, 6: 47, 7: 56, 8: 44, 9: 63}

### (1d.2) Map/reduce

 * Recall that *reduce* applies an operation to 2 elements of the same type, and returns another element of that type. Thus, first thing to do is to map our collection to the type of the output. We cannot use dicts, as dict(list) removes duplictaed keys. We will use list of tuples instead.
 * Then, we have to define a mehtod in the reducer that combines keys. There are two steps:
   * Obtain the keys in the left list
   * Then, check that the key in the second list already exists in the first one

Note the difference with the previous method, based on dictionaries: now, keys are not sorted!!

But, where did we sort the keys in *histSimple*??? Well, we didn´t, but python *dictionary* does that internally for us to speed up things. See the difference in time...

**(1d.3) Map/reduce with pre-sorting**  As shown, the sorting of keys used by a dictionary actually speed up the process. 

However, our *combineKeys* method is creating an array of keys and checking whether a new key is already present in every step. This can be avoid by sorting the initial list first.

Now, computing times get closer. Still, our map/reduce methods are slower, since we cannot use dictionaries...

# Part 2: Spark. Work with RDD and Pair RDD abstractions 

![drawing](https://prateekvjoshi.files.wordpress.com/2015/10/1-main4.png)

# ** Apache Spark**

Apache Spark is an open source cluster computing framework originally developed in the AMPLab at University of California, Berkeley but was later donated to the Apache Software Foundation where it remains today. In contrast to Hadoop's two-stage disk-based MapReduce paradigm, Spark's multi-stage in-memory primitives provides performance up to 100 times faster for certain applications.

![](http://image.slidesharecdn.com/sparkandshark-120620130508-phpapp01/95/spark-and-shark-8-728.jpg?cb=1340197567)

By allowing user programs to load data into a cluster's memory and query it repeatedly, Spark is well-suited to machine learning algorithms.
![](http://spark.apache.org/images/logistic-regression.png)

Spark comes with a number of components that provide flexibility and generality.

<img src="http://spark.apache.org/images/spark-stack.png" alt="Drawing" style="width: 500px;"/>


## In this part, we keep on working on the word-count example, this time with spark. The basic abstraction of Spark is the Resilient Distributed Dataset (RDD):

#### «RDDs are fault-tolerant, parallel data structures that let users explicitly persist intermediate results in memory, control their partitioning to optimize data placement, and manipulate them using a rich set of operators.»

 * Read only, partitioned collection of records (immutable).
 * Stores the transformations used to build a dataset (its linage), instead of the data itself. This property ensures fault-tolerance.
 * User can control partitioning and persistence (caching).
 * RDDs are statically typed.
 * … and yes, everything is written in scala ;p. So you better learn a little bit of it!
 
<img src="http://eng.trueaccord.com/wp-content/uploads/2014/10/scala-logo.png" alt="Drawing" style="width: 200px;"/>

#### We will be trying to understand this abstraction with simple examples, using the [Python API](http://spark.apache.org/docs/latest/api/python/index.html)




### ** (2a) Create a base RDD: parallelize, actions and transformations **
We'll start by generating a base RDD by using a Python list and the `sc.parallelize` method.  Then we'll print(out the type of the base RDD.)

In [89]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [90]:
x

[6, 4, 1, 3, 6, 6, 3, 34, 7, 4]

In [91]:
xr = sc.parallelize(x)
xr

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:480

In [92]:
len(a)

500

In [96]:
ar = sc.parallelize(a)
ar

ParallelCollectionRDD[4] at parallelize at PythonRDD.scala:480

**Nothing has actually happened!**

`parallellize` tells spark to distribute the data, but this is not actually done until we perform some action.

Possible actions include couting, collecting, reducing, taking, etc. Take a look at http://spark.apache.org/docs/latest/programming-guide.html#actions


In [97]:
xr.take(3)

[6, 4, 1]

Apart from actions, we can apply transformations to an RDD. Spark won´t do anything, until an action is performed.  

In [ ]:
map(fun, lista)

In [98]:
xr.map(lambda i: (i, 1, i**2))

PythonRDD[7] at RDD at PythonRDD.scala:48

In [102]:
ar2 = ar.map(lambda i: (i, 1, i**2))

In [104]:
ar2.take(5)

[(2, 1, 4), (1, 1, 1), (5, 1, 25), (4, 1, 16), (4, 1, 16)]

In [106]:
xr.sum()

74

In [107]:
xr.count()

10

In [108]:
xr.mean()

7.4

For instance, we can obtain the length of each word

In [109]:
l

['Hola',
 'buenos',
 'dias',
 'estoy',
 'separando',
 'por',
 'espacios',
 'para',
 'hacer',
 'una',
 'lista']

In [111]:
lr = sc.parallelize(l)

In [113]:
lr.map(lambda w: len(w)).collect()

[4, 6, 4, 5, 9, 3, 8, 4, 5, 3, 5]

In [125]:
lr.filter(lambda w: len(w) > 4) \
  .map(lambda w: len(w)) \
  .filter(lambda i: i%2==1).collect()

[5, 9, 5, 5]

In [122]:
lr.filter(lambda w: len(w) > 4 ).filter(lambda w: len(w)%2!=0).collect()

['estoy', 'separando', 'hacer', 'lista']

In [126]:
import random

random.seed(42)
stupid_words = [random.choice(['holi', 
                               'guapi', 
                               'tocoto', 
                               'chachi', 
                               'lerelele']) for _ in range(500)]

In [127]:
wordsRDD = sc.parallelize(stupid_words) 

### **(2b) Persisting and the RDD lineage**

So far, we have seen that Spark RDDs are *lazy evaluated*, i.e. nothing is actually done until an action is performed. In the RDD, the set of transformations to be applied are remembered: this is known as its *lineage*. It has the important consequence of making Spark RDDs *fault tolerant* automatically.

![](http://images.slideplayer.com/14/4499833/slides/slide_10.jpg) 

It might be interesting to store some intermediate results, though: perhaps because we want to apply several different transformations starting from that point, or because we are going to apply an iterative computation (as is customary in machine learning algorithms). For this, Spark has [several ways of persisting](http://spark.apache.org/docs/latest/programming-guide.html#rdd-persistence)

### **(2c) Partitioning **

One important parameter for parallel collections is the number of partitions to cut the dataset into. Spark will run one task for each partition of the cluster. Typically you want 2-4 partitions for each CPU in your cluster.

To get the number of partitions of an RDD, just use `getNumPartitions()` on your RDD. You can change the partitions during RDD creation (with `parallelize(collection,numPartitions)` or `fromTextFile(file,numPartitions)`), or afterwards with methos like `repartition(), coalesce()`, etc.

In [128]:
wordsRDD.take(5)

['holi', 'holi', 'tocoto', 'guapi', 'guapi']

In [129]:
wordsRDD.getNumPartitions()

4

In [132]:
wordsRDD = wordsRDD.repartition(10)

In [133]:
wordsRDD.getNumPartitions()

10

In [134]:
wordsRDD.coalesce(5)

CoalescedRDD[40] at coalesce at NativeMethodAccessorImpl.java:0

We can see the partitions using [glom()](http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=glom#pyspark.RDD.glom): it retruns an RDD created by coalescing all elements within each partition into a list.

In [144]:
len(wordsRDD.glom().take(2)[1])

55

Partitions are one of the most powerfull concepts in Spark: you can decide how to distribute your data so it can fit in memory, and more importantly, you can perform computations on each partition *before* speaking to other partitions. This can have an enorumous impact on performance

### **(2c) Pair RDDs: *grouping* strategies in Spark**

The next step in writing our word counting program is to create a new type of RDD, called a pair RDD. A pair RDD is an RDD where each element is a pair tuple (k, v) where k is the key and v is the value. In this example, we will create a pair consisting of ('<word>', 1) for each word element in the RDD, as we did in the map/reduce version of the histogram in Python, section (1d.2).

We can create the pair RDD using the map() transformation with a lambda() function to create a new RDD.

In [145]:
wordsRDD.take(5)

['guapi', 'lerelele', 'tocoto', 'guapi', 'chachi']

In [147]:
wordsRDD.map(lambda w: (w,1)).take(5)

[('guapi', 1), ('lerelele', 1), ('tocoto', 1), ('guapi', 1), ('chachi', 1)]

### ** (2c.1) `groupByKey()` approach **
An approach you might first consider (we'll see shortly that there are better ways) is based on using the [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) transformation. As the name implies, the `groupByKey()` transformation groups all the elements of the RDD with the same key into a single list in one of the partitions. There are two problems with using `groupByKey()`:
  + The operation requires a lot of data movement to move all the values into the appropriate partitions.
  + The lists can be very large. Consider a word count of English Wikipedia: the lists for common words (e.g., the, a, etc.) would be huge and could exhaust the available memory in a worker.
 
Use `groupByKey()` to generate a pair RDD of type `('word', iterator)`. Next, sum the iterator using a `map()` transformation.  The result should be a pair RDD consisting of (word, count) pairs.

In [153]:
wordsRDD.map(lambda w: (w,1)).groupByKey().map(lambda i: (i[0], len(i[1]))).collect()

[('lerelele', 101),
 ('holi', 107),
 ('guapi', 106),
 ('tocoto', 98),
 ('chachi', 88)]

In [154]:
wordsRDD.take(3)

['guapi', 'lerelele', 'tocoto']

In [155]:
t1 = wordsRDD.map(lambda w: (w, 1))
t1.take(3)

[('guapi', 1), ('lerelele', 1), ('tocoto', 1)]

In [156]:
t2 = t1.groupByKey()
t2.take(3)

[('lerelele', <pyspark.resultiterable.ResultIterable at 0x7f8fe9a7c400>),
 ('holi', <pyspark.resultiterable.ResultIterable at 0x7f8fe9a7c198>),
 ('guapi', <pyspark.resultiterable.ResultIterable at 0x7f8fe9a76668>)]

In [158]:
t3 = t2.map(lambda i: (i[0], len(i[1])))
t3.collect()

[('lerelele', 101),
 ('holi', 107),
 ('guapi', 106),
 ('tocoto', 98),
 ('chachi', 88)]

In [159]:
wordsRDD.groupByKey().take(3) # groupByKey solo funciona cuando los elementos son tuplas (k,v) 

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 99.0 failed 1 times, most recent failure: Lost task 2.0 in stage 99.0 (TID 189, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 346, in func
    return f(iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 1926, in combine
    merger.mergeValues(iterator)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark

### ** (2c.2)  `reduceByKey` approach **
A better approach is to start from the pair RDD and then use the [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) transformation to create a new pair RDD. 

The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

![](https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/images/reduce_by.png)

In [160]:
wordsRDD.take(3)

['guapi', 'lerelele', 'tocoto']

In [161]:
t1 = wordsRDD.map(lambda w: (w,1))
t1.take(3)

[('guapi', 1), ('lerelele', 1), ('tocoto', 1)]

In [166]:
t1.reduceByKey(lambda acc, i: acc + i).collect()

[('lerelele', 101),
 ('holi', 107),
 ('guapi', 106),
 ('tocoto', 98),
 ('chachi', 88)]

In [168]:
t1.reduceByKey(lambda acc, i: acc + 1).collect() # error habitual, i no es un elemento de la RDD, puede ser un valor intermedio

[('lerelele', 15), ('holi', 19), ('guapi', 18), ('tocoto', 21), ('chachi', 22)]

### ** (2c.3)  `combineByKey` approach: the mother of dragons **

The [combineByKey(createCombiner, mergeValue, mergeCombiners, numPartitions=None)](http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=combinebykey#pyspark.RDD.combineByKey) method is a generic (and powerful!)function to combine the elements for each key using a custom set of aggregation functions.

It turns an RDD[(K, V)] into a result of type RDD[(K, C)], for a “combined type” C. Note that V and C can be different – for example, one might group an RDD of type (Int, Int) into an RDD of type (Int, List[Int]).

Users provide three functions:

#### * createCombiner, which turns a V into a C (e.g., creates a one-element list)
#### * mergeValue, to merge a V into a C (e.g., adds it to the end of a list)
#### * mergeCombiners, to combine two C’s into a single one.

In [ ]:
# let's return the count and the length of words:
wordPairs = wordsRDD.map(lambda w: (w, 1))
wordPairs.combineByKey(
    <fill in>,
    <fill in>,
    <fill in>
).collectAsMap()

In [ ]:
# let's return the count and the length of words:
wordPairs = wordsRDD.<fill in>
wordPairs.combineByKey(
    <fill in>,
    <fill in>,
    <fill in>
).collectAsMap()

In [ ]:
# let's return the count and the list of words:
wordPairs = wordsRDD.map(lambda i:(i,i))
wordPairs.combineByKey(
    <fill in>,
    <fill in>,
    <fill in>
).collectAsMap()

## (2d) Apply word count to a file

### ** (2d.1) `wordCount` function **
First, define a function for word counting.  You should reuse the techniques that have been covered in earlier parts of this lab.  This function should take in an RDD that is a list of words like `wordsRDD` and return a pair RDD that has all of the words and their associated counts.

In [170]:
!curl -v http://www.gutenberg.org/files/100/100-0.txt -o shakespeare.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 152.19.134.47...
* TCP_NODELAY set
* Connected to www.gutenberg.org (152.19.134.47) port 80 (#0)
> GET /files/100/100-0.txt HTTP/1.1
> Host: www.gutenberg.org
> User-Agent: curl/7.55.1
> Accept: */*
> 
< HTTP/1.1 200 OK
< Server: Apache
< Set-Cookie: session_id=f1bf545debe7d59b3101411bb486af75e119080e; Domain=.gutenberg.org; expires=Sat, 21 Apr 2018 11:57:49 GMT; Path=/
< X-Rate-Limiter: zipcat2.php
< Vary: negotiate,accept-encoding
< Last-Modified: Mon, 19 February 2018 17:57:04 GMT
< ETag: "7272f20e"
< X-Zipcat: 5858792 / 5858792 = 1.000
< Accept-Ranges: none
< X-Frame-Options: sameorigin
< X-Connection: Close
< Content-Type: text/plain; charset=UTF-8
< X-Powered-By: 3
< Content-Length: 5858792
< Date: Sat, 21 Apr 2018 11:27:49 GMT
< X-Varni

In [171]:
!head shakespeare.txt

### 2018-05-04

A partir de aquí, el ejercicio de Word Count que hicimos en clase el 4 de Mayo como parte del repaso.

Primero, leemos el archivo. El método sc.textFile crea un RDD de strings, cada una de las cuales representa una línea del archivo

In [1]:
wordListRDD = sc.textFile('data/shakespeare.txt')

In [2]:
wordListRDD

data/shakespeare.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [3]:
wordListRDD.take(3)

['1609', '', 'THE SONNETS']

Recordad que `reducebykey` toma un RDD[(K, V)] y devuelve un RDD[(K, V)]. Es decir, si queremos que el resultado final sea un RDD[(str, int)], cada una de cuyas tuplas representará una palabra (la key) y el número de veces que aparece en el corpus (el value), previamente tenemos que convertir nuestro RDD[str] en un RDD[(str, int)].

Para eso utilizamos un map.

In [4]:
wordListRDD.map(lambda l: l.split()).map(lambda w: (w,1)).take(3)

[(['1609'], 1), ([], 1), (['THE', 'SONNETS'], 1)]

El RDD anterior contiene listas, porque str.split() produce listas. `flatMap` nos permite "concatenar" los resultados de cada invocación de f dentro del `map` para obtener un RDD "plano":

In [5]:
wordListRDD.flatMap(lambda l: l.split()).map(lambda w: (w,1)).take(30)

[('1609', 1),
 ('THE', 1),
 ('SONNETS', 1),
 ('by', 1),
 ('William', 1),
 ('Shakespeare', 1),
 ('1', 1),
 ('From', 1),
 ('fairest', 1),
 ('creatures', 1),
 ('we', 1),
 ('desire', 1),
 ('increase,', 1),
 ('That', 1),
 ('thereby', 1),
 ("beauty's", 1),
 ('rose', 1),
 ('might', 1),
 ('never', 1),
 ('die,', 1),
 ('But', 1),
 ('as', 1),
 ('the', 1),
 ('riper', 1),
 ('should', 1),
 ('by', 1),
 ('time', 1),
 ('decease,', 1),
 ('His', 1),
 ('tender', 1)]

El proceso completo tiene este aspecto:

In [6]:
gruposPalabras = wordListRDD\
    .flatMap(lambda l: l.split())\
    .map(lambda w: (w,1))\
    .reduceByKey(lambda acc, v: acc+v)

In [8]:
gruposPalabras

PythonRDD[10] at RDD at PythonRDD.scala:48

In [7]:
gruposPalabras.take(4)

[('Shakespeare', 38), ('1', 13), ('fairest', 38), ('creatures', 24)]

Podemos ordenar los elementos del RDD con el criterion que queramos usando el parámetro key de sortBy. Aquí extraemos el segundo elemento de cada tupla, el value.

In [9]:
gruposPalabras.sortBy(lambda i: i[1], ascending=False).take(15)

[('the', 23197),
 ('I', 19540),
 ('and', 18263),
 ('to', 15592),
 ('of', 15507),
 ('a', 12516),
 ('my', 10825),
 ('in', 9565),
 ('you', 9059),
 ('is', 7831),
 ('that', 7521),
 ('And', 7068),
 ('not', 6946),
 ('with', 6718),
 ('his', 6218)]

#### Propuesta ejercicio: ordenar, pero solo con palabras con 4 letras

Aquí tenemos que dividir en palabras y, antes de contar, quedarnos con aquellas que cumplen el criterio. Usamos `filter`.

In [16]:
shakespearian_words = sc.textFile('data/shakespeare.txt')\
    .flatMap(lambda line: line.split())

In [17]:
shakespearian_words.take(5)

['1609', 'THE', 'SONNETS', 'by', 'William']

In [27]:
four_letter_words = shakespearian_words.filter(lambda word: len(word) == 4)
four_letter_words.take(10)

['1609',
 'From',
 'That',
 'rose',
 'die,',
 'time',
 'heir',
 'bear',
 'thou',
 'with']

In [28]:
a_pair_rdd = four_letter_words\
    .map(lambda word: (word, 1))
    
a_pair_rdd.take(5)    

[('1609', 1), ('From', 1), ('That', 1), ('rose', 1), ('die,', 1)]

In [29]:
word_counts = a_pair_rdd.reduceByKey(lambda x, y: x + y)

word_counts.take(5)

[('That', 2734), ('rose', 40), ('heir', 68), ('bear', 421), ('thou', 4247)]

No sólo el sortBy de Spark tiene un argumento key que acepta funciones: el `sorted` de Python también lo tiene

In [41]:
my_list = ['holi', 'guapi', 'como', 'te', 'van', 'las', 'cosas']

sorted(my_list, key=lambda word: word[1])

['van', 'las', 'te', 'holi', 'como', 'cosas', 'guapi']

In [32]:
sorted_by_frequency = word_counts.sortBy(lambda tuple_: -tuple_[1])

sorted_by_frequency.take(15)

[('that', 7521),
 ('with', 6718),
 ('your', 6003),
 ('have', 5231),
 ('this', 4760),
 ('thou', 4247),
 ('will', 4047),
 ('That', 2734),
 ('from', 2277),
 ('good', 2046),
 ('what', 2019),
 ('they', 1853),
 ('What', 1814),
 ('thee', 1780),
 ("I'll", 1737)]

In [22]:
help(shakespearian_words.sortBy)

Help on method sortBy in module pyspark.rdd:

sortBy(keyfunc, ascending=True, numPartitions=None) method of pyspark.rdd.PipelinedRDD instance
    Sorts this RDD by the given keyfunc
    
    >>> tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]
    >>> sc.parallelize(tmp).sortBy(lambda x: x[0]).collect()
    [('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]
    >>> sc.parallelize(tmp).sortBy(lambda x: x[1]).collect()
    [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]



Esto las ordena según si son de 4 letras o no: tenemos primero todas las que no y luego las que sí, porque False < True. No nos sirve porque no sabemos dónde tenemos que hacer el corte.

In [24]:
strangely_sorted_words = shakespearian_words.sortBy(lambda word: len(word) == 4)

strangely_sorted_words.take(10)

['THE',
 'SONNETS',
 'by',
 'William',
 'Shakespeare',
 '1',
 'fairest',
 'creatures',
 'we',
 'desire']

Un ejemplo más de cómo podemos ordenar por el criterio que queramos:

In [26]:
sorted_words = shakespearian_words.sortBy(lambda word: len(word), ascending=False)

sorted_words.take(10)

['tragical-comical-historical-pastoral;',
 'honorificabilitudinitatibus;',
 "six-or-seven-times-honour'd",
 'KING_HENRY_VIII|EPILOGUE',
 "Richard!'-'God-a-mercy,",
 'water-flies-diminutives',
 'to-and-fro-conflicting',
 "obligation-'Armigero.'",
 'Castalion-King-Urinal.',
 "that-way-accomplish'd"]

Recordad que las transformations de los RDD son lazy, con lo que si por ejemplo tratamos de abrir un archivo que no existe con textFile (que es una transformation, más o menos) no recibiremos un error hasta que llamemos una action sobre ese RDD.

In [13]:
my_inexistent_file = sc.textFile('asdofiihasdlfunaiasoddufhoasidufnadsliufnadsoim')

In [14]:
my_inexistent_file.take(2)

Py4JJavaError: An error occurred while calling o142.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/dani/repos/pyspark-course/asdofiihasdlfunaiasoddufhoasidufnadsliufnadsoim
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:200)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitio

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return wordListRDD.<fill in>
    
print(wordCount(wordsRDD).collect())
assert sorted(wordCount(wordsRDD).collect()) == [('cat', 2), ('elephant', 1), ('rat', 2)]

### ** (2d.2) Capitalization and punctuation **
Real world files are more complicated than the data we have been using in this lab. Some of the issues we have to address are:
  + Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
  + All punctuation should be removed.
  + Any leading or trailing spaces on a line should be removed.
 
Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [re](https://docs.python.org/2/library/re.html) module to remove any text that is not a letter, number, or space. Reading `help(re.sub)` might be useful.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
import re
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub('[^a-zA-Z0-9 ]','',<fill in>)

assert removePunctuation(" The Elephant's 4 cats. ") == 'the elephants 4 cats'
print(removePunctuation('Hi, you!'))
print(removePunctuation(' No under_score!'))
print(removePunctuation(" The Elephant's 4 cats. "))

### ** (2d.3) Load a text file **
For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply the recently defined `removePunctuation()` function using a `map()` transformation to strip out the punctuation and change all text to lowercase.  Since the file is large we use `take(15)`, so that we only print(15 lines.)

In [ ]:
# Just run this code
import os.path
baseDir = os.path.join('data') # wherever you have put the file 'shakespeare.txt'
fileName = os.path.join(baseDir, 'shakespeare.txt')

shakespeareRDD = (sc
                  .textFile(fileName, 8)
                  .map(removePunctuation))
print('\n'.join(shakespeareRDD)
                .zipWithIndex()  # to (line, lineNum)
                .map(lambda (l, num): '{0}: {1}'.format(num, l))  # to 'lineNum: line'
                .take(15))

### ** (2d.4) Words from lines **
Before we can use the `wordcount()` function, we have to address two issues with the format of the RDD:
  + The first issue is that  that we need to split each line by its spaces.
  + The second issue is we need to filter out empty lines.
 
Apply a transformation that will split each element of the RDD by its spaces. For each element of the RDD, you should apply Python's string [split()](https://docs.python.org/2/library/string.html#string.split) function. You might think that a `map()` transformation is the way to do this, but think about what the result of the `split()` function will be.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
shakespeareWordsRDD = shakespeareRDD.<fill in>
shakespeareWordCount = shakespeareWordsRDD.count()
print(shakespeareWordsRDD.top(5))
print(shakespeareWordCount)
assert (shakespeareWordCount == 927631 or shakespeareWordCount == 928908)

### ** (2d.5) Remove empty elements **
The next step is to filter out the empty elements.  Remove all entries where the word is `''`.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
shakeWordsRDD = shakespeareWordsRDD.<fill in>
shakeWordCount = shakeWordsRDD.count()
print(shakeWordsRDD.take(4))
print(shakeWordCount)
assert shakeWordCount == 882996

### ** (2d.6) Count the words **
We now have an RDD that is only words.  Next, let's apply the `wordCount()` function to produce a list of word counts. We can view the top 15 words by using the `takeOrdered()` action; however, since the elements of the RDD are pairs, we need a custom sort function that sorts using the value part of the pair.

You'll notice that many of the words are common English words (know as stopwords).

Use the `wordCount()` function and `takeOrdered()` to obtain the fifteen most common words and their counts.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
top15WordsAndCounts = <fill in>
print('\n'.join(map(lambda (w, c): '{0}: {1}'.format(w, c), top15WordsAndCounts)))